In [2]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

In [4]:
from dotenv import load_dotenv
import os
load_dotenv()

True

### Q1. Define function description

In [7]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get the weather data (temperature in Celsius) for a given city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city to get the weather for"
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

### Test

In [10]:
!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

--2025-08-29 14:38:56--  https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3495 (3.4K) [text/plain]
Saving to: ‘chat_assistant.py.2’

chat_assistant.py.2 100%[===================>]   3.41K  --.-KB/s    in 0s      

2025-08-29 14:38:56 (18.2 MB/s) - ‘chat_assistant.py.2’ saved [3495/3495]



In [12]:
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [14]:
from chat_assistant import ChatAssistant, ChatInterface, Tools

tools = Tools()
tools.add_tool(get_weather, get_weather_tool)

assistant = ChatAssistant(
    tools=tools,
    chat_interface=ChatInterface(),
    client=client,
    developer_prompt=(
        "You are a helpful assistant. "
        "If the user asks about weather, call get_weather with a city. "
        "If they mention a country, default to a well-known city (e.g., Germany -> Berlin)."
    ),
)
assistant.run()

You: stop


Chat ended.


### Q2. Adding another tool

In [16]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [18]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Set the weather data (temperature in Celsius) for a given city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city to set the weather for"
            },
            "temp": {
                "type": "number",
                "description": "The temperature that you want to change to"
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

### Test

In [21]:
from chat_assistant import ChatAssistant, ChatInterface, Tools

tools = Tools()
tools.add_tool(set_weather, set_weather_tool)

assistant = ChatAssistant(
    tools=tools,
    chat_interface=ChatInterface(),
    client=client,
    developer_prompt=(
        "You are a helpful assistant. "
        "If the user asks to change weather for a given city, call {set_weather} with a given {city} and {temperature}. "
        "If they mention a country, default to a well-known city (e.g., Germany -> Berlin)."
    ),
)
assistant.run()

You: stop


Chat ended.


### Q3. Install FastMCP

In [24]:
!pip install fastmcp

In [26]:
from fastmcp import FastMCP

In [28]:
pip show fastmcp

/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=4224) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Name: fastmcp
Version: 2.11.3
Summary: The fast, Pythonic way to build MCP servers and clients.
Home-page: https://gofastmcp.com
Author: Jeremiah Lowin
Author-email: 
License: 
Location: /opt/anaconda3/lib/python3.12/site-packages
Requires: authlib, cyclopts, exceptiongroup, httpx, mcp, openapi-core, openapi-pydantic, pydantic, pyperclip, python-dotenv, rich
Required-by: 
Note: you may need to restart the kernel to use updated packages.


### Q4. Simple MCP Server

The answer from terminal: Starting MCP server 'Weather server' with transport 'stdio'.
The screenshot of it is attached on GitHub of the HW-Agents

### Q5. Protocol

The answer from terminal: 

{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"24.9"}],"structuredContent":{"result":24.9},"isError":false}}

The screenshot of it is attached on GitHub of the HW-Agents

### Q6. Client

In [60]:
from fastmcp import Client
import weather_server

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        tools = await mcp_client.list_tools()
        print(tools)
        return tools
tools = await main()

[Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta={'_fastmcp': {'tags': []}}), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'},